In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics
from sklearn.model_selection import KFold

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('../assests/scaled_voice_data.csv')

In [5]:
df.head()

,meanfreq,sd,Q75,skew,sfm,mode,meanfun,minfun,maxfun,meandom,maxdom,modindx,gender
0,-0.851530,0.288750,-0.702135,0.045661,0.658078,-0.808182,-1.070300,-1.049225,-1.217722,0.428449,-0.310231,1.490041,male
1,2.084694,-1.169544,1.722539,-0.050675,-1.279251,1.180453,1.367092,0.739336,0.618983,1.463027,1.309144,-0.481520,female
2,-0.517073,0.054301,-1.078278,-0.160767,0.305147,0.249806,-1.231903,0.484349,-0.666264,0.564103,0.108621,1.352695,male
3,0.381459,0.621918,1.226999,-0.221100,0.196662,1.146302,-0.213571,0.697687,0.672624,0.498357,0.636762,-0.582489,male
4,1.109734,-1.411762,0.757406,-0.409960,-1.490037,1.033517,0.629344,0.540973,0.565959,0.660845,1.149370,-0.812012,female


In [6]:
x = df.drop('gender',axis=1)
y = df['gender']

In [7]:
x.head()

,meanfreq,sd,Q75,skew,sfm,mode,meanfun,minfun,maxfun,meandom,maxdom,modindx
0,-0.851530,0.288750,-0.702135,0.045661,0.658078,-0.808182,-1.070300,-1.049225,-1.217722,0.428449,-0.310231,1.490041
1,2.084694,-1.169544,1.722539,-0.050675,-1.279251,1.180453,1.367092,0.739336,0.618983,1.463027,1.309144,-0.481520
2,-0.517073,0.054301,-1.078278,-0.160767,0.305147,0.249806,-1.231903,0.484349,-0.666264,0.564103,0.108621,1.352695
3,0.381459,0.621918,1.226999,-0.221100,0.196662,1.146302,-0.213571,0.697687,0.672624,0.498357,0.636762,-0.582489
4,1.109734,-1.411762,0.757406,-0.409960,-1.490037,1.033517,0.629344,0.540973,0.565959,0.660845,1.149370,-0.812012


In [8]:
y.head()

0      male
1    female
2      male
3      male
4    female
Name: gender, dtype: object

### Perform K-Cross Validation for k = 10

Logistic regression model

In [9]:
def perform_logistic_regression(x_train, y_train, x_test, y_test, i, k, display_stats=False):
    log_reg_model = LogisticRegression(penalty='l2', max_iter=500)
    log_reg_model.fit(x_train, y_train)
    y_pred = log_reg_model.predict(x_test)

    # Accuracy, Precision, Recall, F1-score of Model
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='weighted')
    recall = metrics.recall_score(y_test, y_pred, average='weighted')
    f1_score = metrics.f1_score(y_test, y_pred, average='weighted')

    if display_stats:
        print(f"Statistics for Fold {i} in K-fold (k={k}) 'Logistic Regression' algorithm")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1_score}")
        print()

    return np.array([accuracy, precision, recall, f1_score])

Perform K-corss validation

In [10]:
k = 10
i = 0
k_fold = KFold(n_splits=k)

# Holds Mean of {accuracy, precision, recall, f1_score}
mean_stats = np.zeros(4)

for train_index, test_index in k_fold.split(x):
    x_train, x_test = x.loc[train_index], x.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    stats = perform_logistic_regression(x_train, y_train, x_test, y_test, i:=i+1, k, display_stats=False)
    mean_stats = mean_stats + stats
    
# Take mean of {accuracy, precision, recall, f1_score}
mean_stats = mean_stats / k

print(f"Mean accuracy: {mean_stats[0]}")
print(f"Mean Precision: {mean_stats[1]}")
print(f"Mean Recall: {mean_stats[2]}")
print(f"MeanF1-score: {mean_stats[3]}")

Mean accuracy: 0.970328035778461
Mean Precision: 0.9704880943727809
Mean Recall: 0.970328035778461
MeanF1-score: 0.9703254508389015
